David Giacobbi  
Gonzaga University  
CPSC 222, Spring 2022

# Exploratory Data Analysis: Apple Watch Fitness Data

## Introduction

During my first year of college, fitness has been a critical aspect of my daily life and routine. In a place where nearly everything is in walking distance and I have the free time to workout on an ordered schedule, I realized fitness has a large impact on my daily habits. Moreover, I recently got an Apple Watch over Christmas break, so a detailed and accurate analysis of my fitness data was accessible for the Spring 2022 semester.

Fitness data is a large spectrum, so I wanted to focus my project specifically on how certain day attributes affect health and workout activity. Visualizing and testing trends could help answer a few of the following questions:

* Does the type of weather affect the amount I workout?
* Is my workout intensity influenced by the weather?
* How does the day of the week affect how active I am?
* Am I more active on the weekends when I have more free time or does a structured weekday schedule have a greater effect?

In order to answer these questions, data cannot just be extracted from my Apple Watch. In addition to csv files from my Apple Watch, I will utilize an open-source API to create a JSON file of the weather.

Through this exploratory data analysis, I will be able to get a better understanding of just what influences my fitness habits. Information like this could help me improve my daily schedule to workout more effectively as well as find areas in my schedule where fitness could improve. Furthermore, a focused analysis such as this one could draw more general conclusions about how the workout mindset is influenced by both weather and times during the week.

### Load the Data

In order to perform an extensive analysis of my Apple watch data, I need to gather data from various sources so that I can see what exactly influences my fitness activity and what trends I have created over the past semester. The following datasets will be loaded and cleaned for further analysis:

1. Apple Daily Health data (12/26/2021 - 4/12/2022): csv file
1. Apple Workout data (12/26/2021 - 4/12/2022): csv file
1. MeteoStat Daily Weather data: JSON file
1. Days of the Week data: created from `daily_health_data.csv`

#### Loading the CSV Files

The Apple Watch data needs to be uploaded into `Pandas` dataframes so that they can be properly cleaned for analysis. Since this will be the dataframe that the other data files will be surrounded around, the indexing will be done by date. This can be done with a few quick lines of code below:

In [134]:
import pandas as pd

daily_health_df = pd.read_csv("daily_health_data.csv", index_col="Date")

print(daily_health_df.columns)

Index(['Calories', 'Exercise Time (min)', 'Stand Hours', 'Flights Climbed',
       'Heart Rate', 'Max Heart Rate', 'Avg Heart Rate', 'Rest Heart Rate',
       'Step Count', 'Distance (mi) '],
      dtype='object')


The data from the above csv file was retrieved from an application on my iPhone called [Health Auto Export](https://apps.apple.com/us/app/health-auto-export-json-csv/id1115567069). Using an Apple shortcut, Health Auto Export was able to generate an extensive csv file of various health attributes. Each attribute listed above will be cleaned or deleted, depending on its relative performance. The below attributes are the ones that will be kept for further analysis.

* **Calories**: total active calories burned (kcal)
* **Exercise Time**: total exercise time (min)
* **Flights Climbed**: total flights of stairs climbed
* **Max Heart Rate**: highest heart rate reached (bpm)
* **Avg Heart Rate**: average heart rate throughout day (bpm)
* **Rest Heart Rate**: resting heart rate (bpm)
* **Step Count**: total steps taken
* **Distance**: total active distance covered (mi)
  
  
  
The next Apple Watch csv file is workout centered. This file includes data relating to specific workout instances. Date cannot be used as this file's index as there are multiple workouts logged under the same day. However, the start time of the workout is unique and can be used as the index for this dataset.

In [135]:
workout_df = pd.read_csv("workouts_data.csv", index_col="Start")

print(workout_df.columns)

Index(['Type', 'End', 'Duration', 'Total Energy (kcal)',
       'Active Energy (kcal)', 'Max Heart Rate (bpm)', 'Avg Heart Rate (bpm)',
       'Distance (mi)', 'Avg Speed(mi/hr)', 'Step Count (count)',
       'Step Cadence (spm)', 'Swim Stroke Count (count)',
       'Swim Stroke Cadence (spm)', 'Flights Climbed (count)',
       'Elevation Ascended (ft)', 'Elevation Descended (ft)'],
      dtype='object')


This workout data was also retrieved using the iPhone app [Health Auto Export](https://apps.apple.com/us/app/health-auto-export-json-csv/id1115567069). The Apple shortcut used to create this csv file did not allow for decisions to be made about which attributes to use. Therefore, half of these attributes will be used for data analysis. Below are the attributes that will be used for further analysis:

* **Type**: type of workout completed
* **Duration**: total time of workout (min)
* **Total Energy**: total calories burned in workout (kcal)
* **Max Heart Rate**: highest heart rate reached (bpm)
* **Avg Heart Rate**: average workout heart rate

#### Loading the JSON File

The next data extraction that needs to be completed is the daily weather data for the city of Spokane over the course of this past semester. Found on RapidAPI's website, [MeteoStat's Daily Station API](https://rapidapi.com/meteostat/api/meteostat/) will be used to get a JSON file of the weather data for Spokane from 12/26/2021 - 4/12/2021.

Using [MeteoStat's Station Finder](https://meteostat.net/en/place/us/spokane?t=2022-04-05/2022-04-12&s=KSFF0), Spokane's weather station can be used to create a JSON file with more detailed weather analysis. The code belows uses the above API to get the JSON file, parse through the necessary data, and write it to a `Pandas` dataframe for further analysis:

In [136]:
import json
import requests

weather_url = "https://meteostat.p.rapidapi.com/stations/daily"

weather_key = "48c6248dd3msh5bf7b8a377bd25cp19e73ajsn114ccda534b5"
weather_headers = {"x-rapidapi-key": weather_key}
weather_query = {"station": "KSFF0", "start": "2021-12-26", "end": "2022-04-12", "units": "imperial"}

weather_response = requests.get(url=weather_url, headers=weather_headers, params=weather_query)

# Parse through json object and storel data in dataframe for return
weather_json_obj = json.loads(weather_response.text)
weather_data_list = weather_json_obj["data"]

daily_weather_df = pd.DataFrame(weather_data_list)
daily_weather_df.set_index("date", inplace=True)

print(daily_weather_df)

            tavg  tmin  tmax   prcp  snow   wdir  wspd  wpgt    pres  tsun
date                                                                      
2021-12-26  28.2  25.0  32.0  0.043  None  244.0   5.4  None   997.4  None
2021-12-27  19.4  14.0  25.0  0.024  None  281.0   4.9  None  1006.3  None
2021-12-28  13.3   8.1  17.1  0.000  None  347.0   2.2  None  1010.2  None
2021-12-29  13.1   8.1  17.1  0.000  None  324.0   4.2  None  1012.7  None
2021-12-30  14.9  12.0  17.1  0.177  None    4.0   1.7  None  1006.3  None
...          ...   ...   ...    ...   ...    ...   ...   ...     ...   ...
2022-04-08  48.6  39.2  57.2  0.205  None  207.0  12.6  None  1016.0  None
2022-04-09  40.8  32.0  50.0  0.047  None  228.0  11.7  None  1017.0  None
2022-04-10  35.2  28.4  44.6  0.039  None  218.0   6.3  None  1012.2  None
2022-04-11  35.2  28.4  41.0  0.020  None   53.0  11.9  None  1001.2  None
2022-04-12  36.9  32.0  42.8  0.020  None   59.0  11.1  None  1013.5  None

[108 rows x 10 columns]


In order to compare weather data with health data, the date will be used as the index for clarity. This JSON file provides extensive information about the daily weather; however, some of these attributes do not apply to the cross-analysis with the Apple Watch data. The below attributes will be kept and cleaned for future analysis:

* **tavg**: average temperature (Fahrenheit)
* **tmin**: lowest temperature of the day (Fahrenheit)
* **tmax**: highest temperature of the day (Fahrenheit)
* **prcp**: total precipitation (in)
* **wspd**: average wind speed (mi/hr)

#### Creating the Days of the Week Dataframe

The Days of the Week file is important in weekend v. weekday analysis. In order to create a `Pandas` dataframe of this information, the timestamp of each day and days of week need to be added to an empty dataframe. [GeeksForGeeks](https://www.geeksforgeeks.org/python-pandas-date_range-method/#:~:text=date_range()%20is%20one%20of,return%20a%20fixed%20frequency%20DatetimeIndex.&text=Parameters%3A,Right%20bound%20for%20generating%20dates.) and [Stack Overflow](https://stackoverflow.com/questions/30222533/create-a-day-of-week-column-in-a-pandas-dataframe-using-python) were used to create this dataframe from scratch.

In [137]:
days_of_week_df = pd.DataFrame()

days_of_week_df["Date"] = pd.Series(pd.date_range('2021-12-26', '2022-04-12', freq='D'))
days_of_week_df["Day of Week"] = days_of_week_df["Date"].dt.day_name()

days_of_week_df.set_index("Date", inplace=True)

print(days_of_week_df)

           Day of Week
Date                  
2021-12-26      Sunday
2021-12-27      Monday
2021-12-28     Tuesday
2021-12-29   Wednesday
2021-12-30    Thursday
...                ...
2022-04-08      Friday
2022-04-09    Saturday
2022-04-10      Sunday
2022-04-11      Monday
2022-04-12     Tuesday

[108 rows x 1 columns]


## Data Analysis

Now that all of the necessary data has been loaded into dataframes, the data analysis can proceed. After each dataframe has been combed through for missing values, the cleaned sets can then be visualized, joined, and compared. The next part will take the following steps to get a better grasps of relations among the dataframes:

1. Data Cleaning
1. Data Aggregation
1. Data Visualization
1. Hypothesis Testing

### Data Cleaning

As mentioned in the introduction, some of the datasets have unnecessary attributes that clutter the dataset and missing values that need to be filled. This needs to be taken care of before the data can be joined and compared.

#### Daily Health Dataframe

This dataset will delete the following attribute columns due to irrelevance or lack of data points for further analysis:

* Heart Rate
* Stand Hours

In [138]:
daily_health_df.drop("Heart Rate", axis=1, inplace=True)
daily_health_df.drop("Stand Hours", axis=1, inplace=True)

print(daily_health_df.columns)

Index(['Calories', 'Exercise Time (min)', 'Flights Climbed', 'Max Heart Rate',
       'Avg Heart Rate', 'Rest Heart Rate', 'Step Count', 'Distance (mi) '],
      dtype='object')


Now that the unnecessary columns have been deleted, missing values need to be filled. After looking at the data, a couple of different types of data fills will be used. For this dataframe, the following attributes will use this type of fill:

1. **Exercise Time**: fill with `0` since not enough exercise was completed to log
1. **Flights Climbed**: fill with `0` as days without enough elevation gain did not register a flight climbed
1. **Rest Heart Rate**: fill with `daily_health_df["Rest Heart Rate"].mean()` as middle value since only few days are missing

In [139]:
daily_health_df["Exercise Time (min)"].fillna(0, inplace=True)
daily_health_df["Flights Climbed"].fillna(0, inplace=True)

rest_avg = daily_health_df["Rest Heart Rate"].mean()
daily_health_df["Rest Heart Rate"].fillna(rest_avg, inplace=True)

for i in range(len(daily_health_df.columns)):
    print("Null value column", i, "count:", daily_health_df.iloc[i].isnull().sum())

Null value column 0 count: 0
Null value column 1 count: 0
Null value column 2 count: 0
Null value column 3 count: 0
Null value column 4 count: 0
Null value column 5 count: 0
Null value column 6 count: 0
Null value column 7 count: 0


#### Workout Dataframe

The only cleaning that needs to be completed for the workout dataframe is deletion of columns with non-essential data. Some other columns lack enough data points to be used for statistical analysis. Therefore, the following attributes will be removed from this dataframe:

* End
* Active Energy (kcal)
* Distance (mi)
* Avg Speed (mi/hr)
* Step Count (count)
* Step Cadence (spm)
* Swim Stroke Count (count)
* Swim Stroke Cadence (count)
* Flights Climbed (count)
* Elevation Ascended (ft)
* Elevation Descended (ft)

In [140]:
workout_df.drop("End", axis=1, inplace=True)
workout_df.drop("Active Energy (kcal)", axis=1, inplace=True)
workout_df.drop("Distance (mi)", axis=1, inplace=True)
workout_df.drop("Avg Speed(mi/hr)", axis=1, inplace=True)
workout_df.drop("Step Count (count)", axis=1, inplace=True)
workout_df.drop("Step Cadence (spm)", axis=1, inplace=True)
workout_df.drop("Swim Stroke Count (count)", axis=1, inplace=True)
workout_df.drop("Swim Stroke Cadence (spm)", axis=1, inplace=True)
workout_df.drop("Flights Climbed (count)", axis=1, inplace=True)
workout_df.drop("Elevation Ascended (ft)", axis=1, inplace=True)
workout_df.drop("Elevation Descended (ft)", axis=1, inplace=True)

print(workout_df.columns)
for i in range(len(workout_df.columns)):
    print("Null value column", i, "count:", workout_df.iloc[i].isnull().sum())

Index(['Type', 'Duration', 'Total Energy (kcal)', 'Max Heart Rate (bpm)',
       'Avg Heart Rate (bpm)'],
      dtype='object')
Null value column 0 count: 0
Null value column 1 count: 0
Null value column 2 count: 0
Null value column 3 count: 0
Null value column 4 count: 0


#### Daily Weather Dataframe

Similar to the workout dataframe there is little work that needs to be done in order to prepare it for statistical analysis. There are a few columns that do not have necessary information for visualization, so the following attributes will be removed from the dataframe:

* snow (snow fallen)
* wdir (wind direction)
* wpgt (peak wind gust)
* pres (sea-level air pressure)
* tsun (sunshine total in min)

In [141]:
daily_weather_df.drop("snow", axis=1, inplace=True)
daily_weather_df.drop("wdir", axis=1, inplace=True)
daily_weather_df.drop("wpgt", axis=1, inplace=True)
daily_weather_df.drop("pres", axis=1, inplace=True)
daily_weather_df.drop("tsun", axis=1, inplace=True)

print(daily_weather_df.columns)
for i in range(len(daily_weather_df.columns)):
    print("Null value column", i, "count:", daily_weather_df.iloc[i].isnull().sum())

Index(['tavg', 'tmin', 'tmax', 'prcp', 'wspd'], dtype='object')
Null value column 0 count: 0
Null value column 1 count: 0
Null value column 2 count: 0
Null value column 3 count: 0
Null value column 4 count: 0


### Data Aggregation

In order to properly find trends to visualize and test on, the cleaned dataframes need to be joined and organized by specific attributes. First, the daily health dataframe needs to be joined with the daily weather dataframe. Additionally, the days of the week dataframe needs to be joined with both the daily health and workout dataframe.